# Initializations

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import math
import torch

### Weight Initialization

The aim of weight initialization is to prevent layer activation outputs from exploding or vanishing during the course of a forward pass through a deep neural network. If either occurs, loss gradients will either be too large or too small to flow backwards beneficially, and networks will take longer to converge, if they are even able to do so at all. 

Let's first think about initial weight values in the context of how neural network layers work. Matrix multiplication is the basic operation in a neural network. Here's a quick-and-dirty thought exercise to help us get a conceptual feel for why weight initialization matters in neural nets with several layers. i.e. neural nets where matrix multiplications are performed over and over in succession.

Let's pretend we have a vector `x` that contains some network inputs, and that the matrix `a` represents a layer's weights.

It is the standard practice when training neural networks to ensure that our inputs' values are scaled such that they fall inside such a normal distribution with a mean of 0 and a standard deviation of 1.

In [3]:
num_layers = 100
x = torch.randn(512)

Now it turns out that also initializing the values of layer input weights from the same standard normal distribution is never a good idea. To see why this is, let's run through a quick example where we do just that, and see what happens. 

Say we had a simple hundred-layer network with no activations, in order to complete a single forward pass we'd have to perform a single matrix multiplication between layer inputs and weights at each of the hundred layers, for a grand total of *100* consecutive matrix multiplications.

We can simulate this by multiplying `x` by `a` 100 times in a row.

In [4]:
for i in range(num_layers): 
    a = torch.randn(512,512)
    x = a @ x
x.mean(), x.std()

(tensor(nan), tensor(nan))

Whoa! Somewhere during 100 multiplications, the outputs got so big that even the computer wasn't able to recognize their standard deviation and mean as numbers. We can see exactly how long that took to happen.

In [5]:
x = torch.randn(512)

for i in range(num_layers):
    a = torch.randn(512,512)
    x = a @ x
    if torch.isnan(x.std()): break
i

28

The activation outputs exploded within 29 of our hypothetical network "layers." We clearly initialized our weights (the `a` matrix) to be too large.

Now let's see the other side of the coin: what happens when we initialize network weights to be too small -- we'll scale our initial weights values so that, although still being in a normal distribution with a mean about 0, they have a standard deviation of 0.01.

In [6]:
x = torch.randn(512)

for i in range(num_layers): 
    a = torch.randn(512,512) * 0.01
    x = a @ x
x.mean(), x.std()

(tensor(0.), tensor(0.))

During the course of the above hypothetical forward pass, the activation completely vanished. 

To sum it up, if weights are initialized too large, the network won't learn well. The same happens when weights are initialized too small. How can we find the sweet spot?

Remember that as mentioned above, the math required to complete a forward pass through a neural network entails nothing more than a succession of matrix multiplications. If we have an output `y` that is the product of a matrix multiplication between our input vector `x` and weight matrix `a`, each element $i$ in `y` is defined as $$y_{i} = \sum_{k=0}^{n-1}a_{i,k}x_{k}$$
where $i$ is a given row-index of weight matrix `a`, $k$ is both a given column-index in weight matrix `a` and element-index in input vector `x`, and $n$ is the range or total number of elements in `x`. This can also be defined in Python as
```
y[i] = sum([c*d for c,d in zip(a[i], x)])
```
We can demonstrate that the matrix product of our inputs `x` and weight matrix `a` that we initialized from a standard normal distribution will, on average, have a standard deviation very close to the square root of the number of input connections, or $\sqrt{512}$ in our example.

In [7]:
mean, var = 0.,0.
for i in range(num_layers):
    x = torch.randn(512)
    a = torch.randn(512,512)
    y = a @ x
    mean += y.mean().item()
    var += y.pow(2).mean().item()
mean/num_layers, math.sqrt(var/num_layers)

(0.10345667086541653, 22.59337060971232)

In [8]:
math.sqrt(512)

22.627416997969522

This property isn't surprising when we consider it in terms of how matrix multiplication is defined: in order to calculate `y` we sum 512 products of the element-wise multiplication of one element of the inputs `x` by one column of the weights `a`. In our example where both `x` and `a` are initialized using standard normal distributions, each of these 512 products would have a mean of 0 and standard deviation of 1.

In [9]:
mean, var = 0.,0.
for i in range(num_layers):
    x = torch.randn(1)
    a = torch.randn(1)
    y = a*x
    mean += y.item()
    var += y.pow(2).item()
mean/num_layers, math.sqrt(var/num_layers)

(-0.0020318337575008626, 0.8425986086175008)

It then follows that the *sum* of these 512 products would have a mean of 0, variance of 512, and therefore a standard deviation of $\sqrt{512}$. 

And this is why in our example above we saw our layer outputs exploding after 29 consecutive matrix multiplications. In the case of our simple 100-layer architecture, what we'd like is for each layer's outputs to have a standard deviation of about 1. This conceivably would allow us to repeat matrix multiplications across as many network layers as we want, without activations exploding or vanishing.

If we first scale the weight matrix `a` by dividing all its randomly chosen values by $\sqrt{512}$, the element-wise multiplication that fills in one element of the outputs `y` would now have a variance of $\frac{1}{512}$.

In [10]:
mean, var = 0.,0.
for i in range(num_layers):
    x = torch.randn(1)
    a = torch.randn(1)*math.sqrt(1./512)
    y = a*x
    mean += y.item()
    var += y.pow(2).item()
mean/num_layers, var/num_layers

(-0.0013959996483754367, 0.0018002860597962922)

In [11]:
1/512

0.001953125

This means that the standard deviation of the matrix `y`, which contains each of the 512 values that are generated by way of matrix multiplication of inputs `x` and weights `a`, would be 1. Let's confirm this experimentally.

In [12]:
mean, var = 0.,0.
for i in range(num_layers):
    x = torch.randn(512)
    a = torch.randn(512,512)*math.sqrt(1./512)
    y = a @ x
    mean += y.mean().item()
    var += y.pow(2).mean().item()
mean/num_layers, math.sqrt(var/num_layers)

(-0.0043115697824396195, 0.9988602243337175)

Now we let's re-run our quick and dirty 100-layer network by scaling our weights, which we first choose at random from standard normal distribution inside [-1,1], by $\frac{1}{\sqrt{n}}$, where n is the number of network input connections at a layer, or 512, in our example.

In [13]:
x = torch.randn(512)

for i in range(num_layers):
    a = torch.randn(512,512) * math.sqrt(1./512)
    x = a @ x
x.mean(), x.std()

(tensor(0.0483), tensor(1.2035))

Success! Our layer outputs didn't explode after 100 of our hypothetical layers. Now while at first glance it may seem like at this point we can call it a day. Real-world neural networks aren't quite as simple as our first example may seem to indicate. For the sake of simplicity, activation functions were omitted. However, we'd never do this in real life, as it's thanks to these non-linear activation functions that come at the end of network layers, that neural nets can do such a great job of approximating the intricate functions that can make impressive predictions using real-world inputs, such as the classification of handwriting samples.

Now up until a few years ago, most commonly used activation functions were symmetric about a given value, and had ranges that asymptotically approached values that were plus/minus a certain distance from this midpoint. The hyperbolic tangent and soft sign functions exemplify this class of activations:

<img src= 'images/softsign_tanh.png' width='300'/>

Image source: [Sefik Ilkin Serengil's blog](https://sefiks.com/2017/11/10/softsign-as-a-neural-networks-activation-function/)

Let's add a hyperbolic tangent activation function after each layer our hypothetical 100-layer network, and then see what happens when we use our above weight initialization scheme.

In [14]:
def tanh(x): return torch.tanh(x)

In [15]:
x = torch.randn(512)

for i in range(num_layers): 
    a = torch.randn(512,512) * math.sqrt(1./512)
    x = tanh(a @ x)
x.mean(), x.std()

(tensor(0.0027), tensor(0.0577))

The standard deviation of activation outputs of the 100th layer is down to about 0.1, but thankfully activations haven't vanished. We should be feeling pretty good about the initialization scheme we just discovered from scratch!

As intuitive as this approach may now seem in retrospect, you may be surprised to hear that this was not, in fact, the conventional approach for initializing weight layers as recently as 2010. When Xavier Glorot and Yoshua Bengio published their landmark paper titled ['Understanding the difficulty of training deep feedforward neural networks'](http://proceedings.mlr.press/v9/glorot10a/glorot10a.pdf), the "commonly used heuristic" to which they compared their experiments was that of initializing weights from a uniform distribution in [-1,1] and then scaling by $\frac{1}{\sqrt{n}}$.

It turns out this "standard" approach doesn't actually work that well.

In [16]:
x = torch.randn(512)

for i in range(num_layers): 
    a = torch.Tensor(512,512).uniform_(-1, 1) * math.sqrt(1./512)
    x = tanh(a @ x)
x.mean(), x.std()

(tensor(-3.7805e-26), tensor(1.4189e-24))

Rerunning our 100-layer tanh network with the "standard" weight initialization caused activation gradients to become infinitessimally small -- they're just about as good as vanished.

This poor performance is what spurred Glorot and Bengio to propose their own weight initialization strategy, which they referred to as "normalized initialization" in their paper, and which is now popularly termed "Xavier initialization."

Xavier initialization initializes a layer's weights with values chosen from a random uniform distribution bounded between $$\pm\frac{\sqrt{6}}{\sqrt{n_{i} + n_{i+1}}}$$ where $n_{i}$ is the number of incoming network connections, or "fan-in," to the layer, and $n_{i+1}$ is the number of outgoing network connections from that layer, also known as the "fan-out."

Glorot and Bengio believed that Xavier weight initialization would  maintain the variance of activations and back-propagated gradients all the way up or down a network. They theorized that this would allow for the successful training of deeper networks, as they observed that Xavier initialization enabled a 5-layer network to  maintain a near identical variance of weight gradients across layers. 

<img src= 'images/xavier_init.png' width='500'/>

Image source: [Glorot & Bengio](http://proceedings.mlr.press/v9/glorot10a/glorot10a.pdf)

Using "standard" initialization brought about a much bigger gap between weight gradients of the network's lower layers, which were higher, and those of its top-most layers, which were approaching zero.

<img src= 'images/standard_init.png' width='500'/>

Image source: [Glorot & Bengio](http://proceedings.mlr.press/v9/glorot10a/glorot10a.pdf)

To drive the point home, they demonstrated that networks thus initialized achieved substationally quicker convergence and higher accuracy on the [CIFAR-10 image classification task](https://www.cs.toronto.edu/~kriz/cifar.html).

We can re-run our 100-layer tanh "network" once more, this time using Xavier initialization:

In [17]:
def xavier(m,h): 
    return torch.Tensor(m, h).uniform_(-1, 1)*math.sqrt(6./(m+h))

In [18]:
x = torch.randn(512)

for i in range(num_layers):
    a = xavier(512, 512)
    x = tanh(a @ x)
x.mean(), x.std()

(tensor(-0.0003), tensor(0.0665))

We can see that in our experimental network, Xavier initialization performs pretty identical to our home-grown method that we derived above, where we sampled values from a random normal distribution and scaled by the square root of number of incoming network connections, $n$.

Conceptually, when we are using activation functions that are symmetric about zero and have outputs inside [-1,1], such as softsign and the hyperbolic tangent, it makes sense that we'd want our network layer activation outputs to have a mean of 0 and a standard deviation around 1, on average. And this is precisely what our method and the Xavier method enable.

But what if we're using ReLU activation functions? Would it still make sense to want to scale random initial weight values by the same factor?

<img src= 'images/relu.png' width='300'/>

Image source: [Kanchan Sarkar's blog](https://medium.com/@kanchansarkar/relu-not-a-differentiable-function-why-used-in-gradient-based-optimization-7fef3a4cecec)

Let's use a ReLU activation instead of tanh in one of our hypothetical network's layers and observe that the expected standard deviation of outputs would be.

In [19]:
def relu(x): return x.clamp_min(0.)

In [20]:
mean, var = 0.,0.
for i in range(num_layers):
    x = torch.randn(512)
    a = torch.randn(512,512)
    y = relu(a @ x)
    mean += y.mean().item()
    var += y.pow(2).mean().item()
mean/num_layers, math.sqrt(var/num_layers)

(8.942873458862305, 15.893485235303322)

It turns out that when using a ReLU activation, a single layer will, on average have standard deviation that's very close to the square root of the number of input connections *divided by two*, or $\sqrt{\frac{512}{2}}$ in our example.

In [21]:
math.sqrt(512/2)

16.0

Scaling the values in our weight matrix `a` by this number will give our ReLU layer a standard deviation of 1 on average.

In [22]:
mean, var = 0.,0.
for i in range(num_layers):
    x = torch.randn(512)
    a = torch.randn(512,512) * math.sqrt(2/512)
    y = relu(a @ x)
    mean += y.mean().item()
    var += y.pow(2).mean().item()
mean/num_layers, math.sqrt(var/num_layers)

(0.5638230794668198, 0.9960980748101388)

As we showed before, keeping the standard deviation of layers' activations around 1 will allow us to stack several more layers in a deep neural network without gradients exploding or vanishing.

In fact, this exploration into how to best scale ReLU activations in order to prevent gradients from exploding or vanishing is what motivated Kaiming He et. al. to in 2015 [propose their own initialization scheme](https://arxiv.org/pdf/1502.01852.pdf) that's tailored for deep neural nets that use asymmetric non-linear activations like ReLU.

He et. al. demonstrated that deep networks (e.g a 22-layer CNN) would converge much earlier if the following input weight initialization strategy is employed:

1. Create a tensor with the dimensions appropriate for a weight matrix at a given layer.
2. Populate the tensor with numbers randomly chosen from a standard distribution.
3. Multiply each randomly chosen number by $\sqrt{\frac{2}{n}}$ where $n$ is the number of incoming connections coming into a given layer from the previous layer's output (also known as the "fan-in").
4. Bias tensors are initialized to zero.

In regards to training even deeper networks that used ReLUs, He et. al. found that a 30-layer CNN using Xavier initialization stalled completely and didn't learn at all. However, when the same network was initialized as recommended above it did, in fact, begin to learn and converge.

We can follow the above directions to implement our own version of Kaiming initialization and verify that it can indeed prevent activation outputs from both exploding or vanishing if we use ReLU instead of tanh at all layers of our hypothetical 100-layer network.

In [23]:
def kaiming(m,h): 
    return torch.randn(m,h)*math.sqrt(2./m)

In [24]:
x = torch.randn(512)

for i in range(num_layers): 
    a = kaiming(512, 512)
    x = relu(a @ x)
x.mean(), x.std()

(tensor(0.4563), tensor(0.6684))

Using Kaiming to initialize weights indeed prevents activation outputs from exploding or vanishing, even after 100 linear layers, each with a ReLU activation! 

Let's see what happens if we were to use Xavier initialization, instead.

In [25]:
x = torch.randn(512)

for i in range(num_layers):
    a = xavier(512, 512)
    x = relu(a @ x)
x.mean(), x.std()

(tensor(4.4963e-16), tensor(6.5360e-16))

Ouch! When using Xavier to initialize weights in our hypothetical 100-layer network with ReLUs, activation gradients almost completely vanish!